# 1. データサイエンティストによるノートブックでの試行錯誤

データが蓄積され取得できるようになったら、データサイエンティストはEDA（探索的データ解析）を行い、モデルを構築し、評価します。
本ノートブックでは、データサイエンティストによるモデル構築コードを提示します。
以降のノートブックで、作成されたスクリプトのモジュール化を行なっていきます。


## 実験内容

下記のノートブックと同様の実験を行います。

https://github.com/aws-samples/aws-ml-jp/blob/main/mlops/step-functions-data-science-sdk/model-train-evaluate-compare/step_functions_mlworkflow_scikit_learn_data_processing_and_model_evaluation_with_experiments.ipynb

>このノートブックで使用するデータは Census-Income KDD Dataset です。このデータセットから特徴量を選択し、データクレンジングを実施し、二値分類モデルの利用できる形にデータを変換し、最後にデータを学習用とテスト用に分割します。このノートブックではロジスティック回帰モデルを使って、国勢調査の回答者の収入が 5万ドル以上か 5万ドル未満かを予測します。このデータセットはクラスごとの不均衡が大きく、ほとんどのデータに 5万ドル以下というラベルが付加されています。

## 前提：データは事前に dataset/ に手動で格納しておく

データを以下のサイトから入手し、 dataset ディレクトリに配置してください。

https://archive.ics.uci.edu/ml/datasets/Census-Income+%28KDD%29

./dataset/census-income.csv(101.5MB)

## データサイエンティストによる、モデル構築
データサイエンティストがEDAを行なったあと、ノートブック上でモデルの構築、評価を行なった場合を想定します。

このスクリプトでは、以下の処理が実行されます。

* 重複データやコンフリクトしているデータの削除
* ターゲット変数 income 列をカテゴリ変数から 2つのラベルを持つ列に変換
* age と num persons worked for employer をビニングして数値からカテゴリ変数に変換
* 連続値であるcapital gains, capital losses, dividends from stocks を学習しやすいようスケーリング
* education, major industry code, class of workerを学習しやすいようエンコード
* データを学習用とテスト用に分割し特徴量とラベルの値をそれぞれ保存

## コードの詳細

以下、69行（空行含む）
* ライブラリ読み込み：7行
* 空行：9行
* コメント：11行
* コード実行：19行
* コード実行の改行：23行

In [ ]:
# Import the latest sagemaker, stepfunctions and boto3 SDKs
import sys

!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install -qU pandas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.compose import make_column_transformer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score

### データ読み込み
columns = [
    "age",
    "education",
    "major industry code",
    "class of worker",
    "num persons worked for employer",
    "capital gains",
    "capital losses",
    "dividends from stocks",
    "income",
]
class_labels = [" - 50000.", " 50000+."]

df = pd.read_csv("./dataset/census-income.csv")
df = df[columns]

### 前処理
#重複データやコンフリクトしているデータの削除
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.replace(class_labels, [0, 1], inplace=True)

#ターゲット変数 income 列をカテゴリ変数から 2つのラベルを持つ列に変換
negative_examples, positive_examples = np.bincount(df["income"])
#データを学習用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(df.drop("income", axis=1), df["income"], test_size=0.2)

preprocess = make_column_transformer(
    #age と num persons worked for employer をビニングして数値からカテゴリ変数に変換
    (
        KBinsDiscretizer(encode="onehot-dense", n_bins=10),
        ["age", "num persons worked for employer"],
    ),
    #連続値であるcapital gains, capital losses, dividends from stocks を学習しやすいようスケーリング
    (
        StandardScaler(),
        ["capital gains", "capital losses", "dividends from stocks"],
    ),
    #education, major industry code, class of workerを学習しやすいようエンコード
    (
        OneHotEncoder(sparse=False, handle_unknown='ignore'),
        ["education", "major industry code", "class of worker"],
    ),
)
X_train = preprocess.fit_transform(X_train)
X_test = preprocess.transform(X_test)

### 学習
model = LogisticRegression(class_weight="balanced", solver="lbfgs", C=float(1.0), verbose=1)
model.fit(X_train, y_train)

### 推論
predictions = model.predict(X_test)

### 評価
report_dict = classification_report(y_test, predictions, output_dict=True)
report_dict["accuracy"] = accuracy_score(y_test, predictions)
report_dict["roc_auc"] = roc_auc_score(y_test, predictions)
print(report_dict)

ノートブックのインタラクティブ性は、EDAやモデルプロトタイプなどの初期の試行錯誤には大変便利です。
一方で、モジュール化されていないコードや記録されていないコードや、本番運用を見据えると、後のコード本番化、リファクタリングなどの工数を増加や、テストの難しさによる品質確保が難しいといった懸念もあります。

試行錯誤の柔軟性を確保しつつ、モジュール化されたコードをきちんと記録していくことが、コードの品質向上と、本番導入の迅速化には重要になります。
以降のノートブックでは、実験を支援するパイプラインを準備し、ノートブックをモジュール化していく例をみていきます。

## [参考] 詰め込んだ場合、以下の23行で完了

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
df = pd.read_csv("./dataset/census-income.csv")
df = df[["age","education","major industry code","class of worker","num persons worked for employer","capital gains","capital losses","dividends from stocks","income",]]
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.replace([" - 50000.", " 50000+."], [0, 1], inplace=True)
X_train, X_test, y_train, y_test = train_test_split(df.drop("income", axis=1), df["income"], test_size=0.2)
preprocess = make_column_transformer((KBinsDiscretizer(encode="onehot-dense", n_bins=10),["age", "num persons worked for employer"],),(StandardScaler(),["capital gains", "capital losses", "dividends from stocks"],),(OneHotEncoder(sparse=False, handle_unknown='ignore'),["education", "major industry code", "class of worker"],),)
X_train = preprocess.fit_transform(X_train)
X_test = preprocess.transform(X_test)
model = LogisticRegression(class_weight="balanced", solver="lbfgs", C=float(1.0), verbose=1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
report_dict = classification_report(y_test, predictions, output_dict=True)
report_dict["accuracy"] = accuracy_score(y_test, predictions)
report_dict["roc_auc"] = roc_auc_score(y_test, predictions)
print(report_dict)